In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


from utils import feature_engineering

In [ ]:
df_credit = pd.read_csv('saved/preprocessed_bank_data.csv')
X_processed = feature_engineering(df_credit)


from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

X = X_processed.drop(columns=['RiskScore'])
y = X_processed['RiskScore']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1)

cat = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.1,
        depth=6,
        l2_leaf_reg=3,
        random_state=42,
        verbose=0
    )

cat.fit(X_train, y_train,  cat_features=["EmploymentStatus", 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'])

cat.save_model(f"saved/cat_model.cbm", format="cbm")



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = cat.predict(X_test)
    
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
user_input = {
  "ApplicationDate": 6,
  "Age": 50,
  "CreditScore": 500,
  "EmploymentStatus": "Employed", #['Employed', 'Self-Employed', 'Unemployed']
  "EducationLevel": "High School", #['Master', 'Associate', 'Bachelor', 'High School', 'Doctorate']
  "LoanAmount": 120000,
  "LoanDuration": 60,
  "MaritalStatus": "Widowed",#['Married', 'Single', 'Divorced', 'Widowed']
  "NumberOfDependents": 1,
  "HomeOwnershipStatus": "Rent",#['Own', 'Mortgage', 'Rent', 'Other']
  "MonthlyDebtPayments": 700000,
  "CreditCardUtilizationRate": 0.6,
  "NumberOfOpenCreditLines": 2,
  "NumberOfCreditInquiries": 1,
  "DebtToIncomeRatio": 0.1,
  "BankruptcyHistory": 0,
  "LoanPurpose": "Debt Consolidation", #['Home', 'Debt Consolidation', 'Education', 'Other', 'Auto']
  "PreviousLoanDefaults": 0,
  "PaymentHistory": 24,
  "LengthOfCreditHistory": 18,
  "SavingsAccountBalance": 20000,
  "CheckingAccountBalance": 15000,
  "TotalAssets": 250000,
  "TotalLiabilities": 20000000,
  "MonthlyIncome": 11000,
  "UtilityBillsPaymentHistory": 0.5,
  "JobTenure": 12,
  "TotalDebtToIncomeRatio": 0.22,
}
user_input = feature_engineering(pd.DataFrame([user_input]))

cat.predict(user_input)